In [1]:
# replace intense-reactor-425120-g7 with your own project id
%env GOOGLE_CLOUD_PROJECT = intense-reactor-425120-g7

env: GOOGLE_CLOUD_PROJECT=intense-reactor-425120-g7


In [2]:
import matplotlib.pyplot as plt
import pandas as pd
from google.colab import files
from google.cloud import bigquery

import missingno as msno
import matplotlib.pyplot as plt

import numpy as np

from google.colab import auth
auth.authenticate_user()
print('Authenticated')

%load_ext google.colab.data_table

Authenticated


In [3]:
#@title Ventilation Query
# https://github.com/MIT-LCP/eicu-code/issues/82
# https://github.com/nus-mornin-lab/oxygenation_kc/blob/master/data-extraction/eICU/eicu_oxygen_therapy.sql
%%bigquery vent_df

DROP TABLE IF EXISTS `intense-reactor-425120-g7.eicu1_pulseOx.eicu_vents`;
CREATE TABLE `intense-reactor-425120-g7.eicu1_pulseOx.eicu_vents` AS

WITH respchart AS (
	SELECT *
	FROM `physionet-data.eicu_crd.respiratorycharting`
)

, nursechart AS (
	SELECT *
	FROM `physionet-data.eicu_crd.nursecharting`
)

, pat AS (
	SELECT *
	FROM `physionet-data.eicu_crd.patient`
)


-- Extract the type of oxygen therapy.
-- The categories are invasive ventilation,
-- noninvasive ventilation, and supplemental oxygen.
-- `oxygen_therapy_type = -1` indicates oxygen therapy,
-- i.e. more oxygen than in room air is administered.
, ventsettings0 AS (
	SELECT patientunitstayid AS icustay_id
		, charttime
		, CASE

			-- Invasive ventilation
			WHEN
				string IN (
					'plateau pressure',
					'postion at lip',
					'position at lip',
					'pressure control'
				)
				OR string LIKE '%set vt%'
				OR string LIKE '%sputum%'
				OR string LIKE '%rsbi%'
				OR string LIKE '%tube%'
				OR string LIKE '%ett%'
				OR string LIKE '%endotracheal%'
				OR string LIKE '%tracheal suctioning%'
				OR string LIKE '%tracheostomy%'
				OR string LIKE '%reintubation%'
				OR string LIKE '%assist controlled%'
				OR string LIKE '%volume controlled%'
				OR string LIKE '%pressure controlled%'
				OR string LIKE '%trach collar%'
			THEN 4

			-- Noninvasive ventilation
			WHEN
				string IN (
					'bi-pap',
					'ambubag'
				)
				OR string LIKE '%ipap%'
				OR string LIKE '%niv%'
				OR string LIKE '%epap%'
				OR string LIKE '%mask leak%'
				OR string LIKE '%volume assured%'
				OR string LIKE '%non-invasive ventilation%'
				OR string LIKE '%cpap%'
			THEN 3

			-- Either invasive or noninvasive ventilation:
			WHEN
				string IN (
					'flowtrigger',
					'peep',
					'tv/kg ibw',
					'mean airway pressure',
					'peak insp. pressure',
					'exhaled mv',
					'exhaled tv (machine)',
					'exhaled tv (patient)',
					'flow sensitivity',
					'peak flow',
					'f total',
					'pressure to trigger ps',
					'adult con setting set rr',
					'adult con setting set vt',
					'vti',
					'exhaled vt',
					'adult con alarms hi press alarm',
					'mve',
					'respiratory phase',
					'inspiratory pressure, set',
					'a1: high exhaled vt',
					'set fraction of inspired oxygen (fio2)',
					'insp flow (l/min)',
					'adult con setting spont exp vt',
					'spont tv',
					'pulse ox results vt',
					'vt spontaneous (ml)',
					'peak pressure',
					'ltv1200',
					'tc'
				)
				OR (
					string LIKE '%vent%'
					AND NOT string LIKE '%hyperventilat%'
				)
				OR string LIKE '%tidal%'
				OR string LIKE '%flow rate%'
				OR string LIKE '%minute volume%'
				OR string LIKE '%leak%'
				OR string LIKE '%pressure support%'
				OR string LIKE '%peep%'
				OR string LIKE '%tidal volume%'
			THEN 2

			-- Supplemental oxygen:
			WHEN
				string IN (
					't-piece',
					'blow-by',
					'oxyhood',
					'nc',
					'oxymizer',
					'hfnc',
					'oximizer',
					'high flow',
					'oxymask',
					'nch',
					'hi flow',
					'hiflow',
					'hhfnc',
					'nasal canula',
					'face tent',
					'high flow mask',
					'aerosol mask',
					'venturi mask',
					'cool aerosol mask',
					'simple mask',
					'face mask'
				)
				OR string LIKE '%nasal cannula%'
				OR string LIKE '%non-rebreather%'
				OR string LIKE '%nasal mask%'
				OR string LIKE '%face tent%'
			THEN 1

			-- Oxygen therapy but unknown what type:
			WHEN
				string IN (
					'pressure support',
					'rr spont',
					'ps',
					'insp cycle off (%)',
					'trach mask/collar'
				)
				OR string LIKE '%spontaneous%'
				OR string LIKE '%oxygen therapy%'
			THEN 0

			-- Supplemental oxygen therapy,
			-- i.e. more oxygen than in room air is administered.
			WHEN
				string IN (
					'lpm o2'
				)
			THEN -1

			ELSE NULL

		END AS oxygen_therapy_type
		, activeUponDischarge
	FROM (

		SELECT patientunitstayid
			, nursingChartOffset AS charttime
			, LOWER(nursingchartvalue) AS string
			, NULL AS activeUponDischarge
		FROM nursechart

		UNION ALL

		SELECT patientunitstayid
			, respchartoffset AS charttime
			, LOWER(respchartvaluelabel) AS string
			, NULL AS activeUponDischarge
		FROM respchart

		UNION ALL

		-- Oxygen device from respchart
		SELECT patientunitstayid
			, respchartoffset AS charttime
			, LOWER(respchartvalue) AS string
			, NULL AS activeUponDischarge
		FROM respchart
		WHERE LOWER(respchartvaluelabel) IN (
			'o2 device',
			'respiratory device',
			'ventilator type',
			'oxygen delivery method'
    	)

    	UNION ALL

    	-- The treatment table also contains info on oxygen therapy.
    	SELECT patientunitstayid
			, treatmentoffset AS charttime
			, LOWER(treatmentstring) AS string
			, activeUponDischarge
		FROM `physionet-data.eicu_crd.treatment`
	)
	WHERE charttime >= -60

	UNION ALL

	-- The following indicates oxygen therapy but unclear what type.
	SELECT patientunitstayid AS icustay_id
		, nursingchartoffset AS charttime
		, -1 AS oxygen_therapy_type
		, NULL AS activeUponDischarge
	FROM nursechart
	WHERE nursingchartoffset >= -60
		AND nursingchartcelltypevallabel = 'O2 L/%'
		AND SAFE_CAST(nursingChartValue AS INT64) > 0
		AND SAFE_CAST(nursingChartValue AS INT64) <= 100

	UNION ALL

	-- fraction of inspired oxygen (fiO2) outside of [.2, .22] and [20, 22]
	-- indicates oxygen therapy.
	SELECT patientunitstayid AS icustay_id
		, respchartoffset AS charttime
		, CASE
			WHEN SAFE_CAST(respchartvalue AS FLOAT64) <= 1 AND SAFE_CAST(respchartvalue AS FLOAT64) > .22 THEN -1
			WHEN SAFE_CAST(respchartvalue AS FLOAT64) > 22 THEN -1
			ELSE 0
		END AS oxygen_therapy_type
		, NULL AS activeUponDischarge
	FROM respchart
	WHERE respchartoffset >= -60
		AND LOWER(respchartvaluelabel) IN ('fio2', 'fio2 (%)')
		AND (
			SAFE_CAST(respchartvalue AS FLOAT64) < .2
			OR (
				SAFE_CAST(respchartvalue AS FLOAT64) > .22
				AND SAFE_CAST(respchartvalue AS FLOAT64) < 20
			)
			OR SAFE_CAST(respchartvalue AS FLOAT64) > 22
		)
)


-- Ensure charttime is unique
, ventsettings AS (
	SELECT icustay_id
		, charttime
		, MAX(oxygen_therapy_type) AS oxygen_therapy_type
		, MAX(activeUponDischarge) AS activeUponDischarge
		, COUNT(CASE WHEN oxygen_therapy_type = -1 THEN 1 END) > 0 AS supp_oxygen
	FROM ventsettings0
	-- If oxygen_therapy_type is NULL,
	-- then the record does not correspond with oxygen therapy.
	WHERE oxygen_therapy_type IS NOT NULL
	GROUP BY icustay_id, charttime
)


, vd0 as
(
  select
    *
    -- this carries over the previous charttime which had an oxygen therapy event
    , LAG(CHARTTIME, 1) OVER (partition by icustay_id order by charttime)
	as charttime_lag
  from ventsettings
)
, vd1 as
(
  select
      icustay_id
      , charttime
      , oxygen_therapy_type
      , activeUponDischarge
      , supp_oxygen

      -- If the time since the last oxygen therapy event is more than 24 hours,
	-- we consider that ventilation had ended in between.
	-- That is, the next ventilation record corresponds to a new ventilation session.
      , CASE
		WHEN charttime - charttime_lag > 24*60 THEN 1
		WHEN charttime_lag IS NULL THEN 1 -- No lag can be computed for the very first record
		ELSE 0
	END AS newvent
  -- use the staging table with only oxygen therapy records from chart events
  FROM vd0
)
, vd2 as
(
  select vd1.*
  -- create a cumulative sum of the instances of new ventilation
  -- this results in a monotonic integer assigned to each instance of ventilation
  , SUM( newvent )
      OVER ( partition by icustay_id order by charttime )
    as ventnum
  from vd1
)

--- now we convert CHARTTIME of ventilator settings into durations
-- create the durations for each oxygen therapy instance
-- We only keep the first oxygen therapy instance
, vd3 AS
(
	SELECT icustay_id
		, ventnum
		, CASE
			-- If activeUponDischarge, then the unit discharge time is vent_end
			WHEN (
				MAX(activeUponDischarge)
				-- vent_end cannot be later than the unit discharge time.
				-- However, unitdischargeoffset often seems too low.
				-- So, we only use it if it yields and extension of the
				-- ventilation time from ventsettings.
				AND MAX(charttime)+60 < MAX(pat.unitdischargeoffset)
			)
			THEN MAX(pat.unitdischargeoffset)
			-- End time is currently a charting time
			-- Since these are usually recorded hourly, ventilation is actually longer.
			-- We therefore add 60 minutes to the last time.
			ELSE MAX(charttime)+60
		END AS vent_end
		, MIN(charttime) AS vent_start
		, MAX(oxygen_therapy_type) AS oxygen_therapy_type
		, MAX(supp_oxygen) AS supp_oxygen
	FROM vd2
		LEFT JOIN pat
		ON vd2.icustay_id = pat.patientunitstayid
	GROUP BY icustay_id, ventnum
)


select vd3.*
	-- vent_duration is in hours.
	, (vent_end - vent_start) / 60 AS vent_duration
	, MIN(vent_start) OVER(PARTITION BY icustay_id) AS vent_start_first
from vd3

Query is running:   0%|          |

In [4]:
#@title Charlson Comorbidity Query
# https://github.com/theonesp/vol_leak_index/blob/master/eicu_vli/analysis/sql/charlson_score.sql
%%bigquery cci

DROP TABLE IF EXISTS `intense-reactor-425120-g7.eicu1_pulseOx.eicu_charlson`;
CREATE TABLE `intense-reactor-425120-g7.eicu1_pulseOx.eicu_charlson` AS

WITH
  t1 AS (
  SELECT
    s.patientunitstayid,
    MAX (CASE
        WHEN ph.pasthistorypath IN ( 'notes/Progress Notes/Past History/Organ Systems/Hematology/Oncology (R)/Cancer/Metastases/other', 'notes/Progress Notes/Past History/Organ Systems/Hematology/Oncology (R)/Cancer/Metastases/brain', 'notes/Progress Notes/Past History/Organ Systems/Hematology/Oncology (R)/Cancer/Metastases/carcinomatosis', 'notes/Progress Notes/Past History/Organ Systems/Hematology/Oncology (R)/Cancer/Metastases/nodes', 'notes/Progress Notes/Past History/Organ Systems/Hematology/Oncology (R)/Cancer/Metastases/lung', 'notes/Progress Notes/Past History/Organ Systems/Hematology/Oncology (R)/Cancer/Metastases/intra-abdominal', 'notes/Progress Notes/Past History/Organ Systems/Hematology/Oncology (R)/Cancer/Metastases/bone', 'notes/Progress Notes/Past History/Organ Systems/Hematology/Oncology (R)/Cancer/Metastases/liver') THEN 6
      ELSE
      0
    END
      ) AS mets6,
    MAX (CASE
        WHEN ph.pasthistorypath = 'notes/Progress Notes/Past History/Organ Systems/Infectious Disease (R)/AIDS/AIDS' THEN 6
      ELSE
      0
    END
      ) AS aids6,
    MAX (CASE
        WHEN ph.pasthistorypath IN ( 'notes/Progress Notes/Past History/Organ Systems/Gastrointestinal (R)/Cirrhosis/UGI bleeding', 'notes/Progress Notes/Past History/Organ Systems/Gastrointestinal (R)/Cirrhosis/varices', 'notes/Progress Notes/Past History/Organ Systems/Gastrointestinal (R)/Cirrhosis/coma', 'notes/Progress Notes/Past History/Organ Systems/Gastrointestinal (R)/Cirrhosis/jaundice', 'notes/Progress Notes/Past History/Organ Systems/Gastrointestinal (R)/Cirrhosis/ascites', 'notes/Progress Notes/Past History/Organ Systems/Gastrointestinal (R)/Cirrhosis/encephalopathy') THEN 3
      ELSE
      0
    END
      ) AS liver3,
    MAX (CASE
        WHEN ph.pasthistorypath IN ( 'notes/Progress Notes/Past History/Organ Systems/Neurologic/Strokes/multiple/multiple', 'notes/Progress Notes/Past History/Organ Systems/Neurologic/Strokes/stroke - remote', 'notes/Progress Notes/Past History/Organ Systems/Neurologic/Strokes/stroke - within 5 years', 'notes/Progress Notes/Past History/Organ Systems/Neurologic/Strokes/stroke - within 2 years', 'notes/Progress Notes/Past History/Organ Systems/Neurologic/Strokes/stroke - date unknown', 'notes/Progress Notes/Past History/Organ Systems/Neurologic/Strokes/stroke - within 6 months') THEN 2
      ELSE
      0
    END
      ) AS stroke2,
    MAX (CASE
        WHEN ph.pasthistorypath IN ( 'notes/Progress Notes/Past History/Organ Systems/Renal  (R)/Renal Insufficiency/renal insufficiency - creatinine 1-2', 'notes/Progress Notes/Past History/Organ Systems/Renal  (R)/Renal Insufficiency/renal insufficiency - creatinine 3-4', 'notes/Progress Notes/Past History/Organ Systems/Renal  (R)/Renal Insufficiency/renal insufficiency - creatinine > 5', 'notes/Progress Notes/Past History/Organ Systems/Renal  (R)/Renal Insufficiency/renal insufficiency - baseline creatinine unknown', 'notes/Progress Notes/Past History/Organ Systems/Renal  (R)/Renal Insufficiency/renal insufficiency - creatinine 4-5', 'notes/Progress Notes/Past History/Organ Systems/Renal  (R)/Renal Insufficiency/renal insufficiency - creatinine 2-3', 'notes/Progress Notes/Past History/Organ Systems/Renal  (R)/Renal Failure/renal failure - peritoneal dialysis', 'notes/Progress Notes/Past History/Organ Systems/Renal  (R)/Renal Failure/renal failure- not currently dialyzed', 'notes/Progress Notes/Past History/Organ Systems/Renal  (R)/Renal Failure/renal failure - hemodialysis') THEN 2
      ELSE
      0
    END
      ) AS renal2,
    MAX (CASE
        WHEN ph.pasthistorypath IN ( 'notes/Progress Notes/Past History/Organ Systems/Endocrine (R)/Insulin Dependent Diabetes/insulin dependent diabetes', 'notes/Progress Notes/Past History/Organ Systems/Endocrine (R)/Non-Insulin Dependent Diabetes/non-medication dependent', 'notes/Progress Notes/Past History/Organ Systems/Endocrine (R)/Non-Insulin Dependent Diabetes/medication dependent') THEN 1
      ELSE
      0
    END
      ) AS dm,
    MAX (CASE
        WHEN ph.pasthistorypath IN ( 'notes/Progress Notes/Past History/Organ Systems/Hematology/Oncology (R)/Cancer Therapy/Chemotherapy/Anthracyclines (adriamycin, daunorubicin)', 'notes/Progress Notes/Past History/Organ Systems/Hematology/Oncology (R)/Cancer/Cancer-Primary Site/bone', 'notes/Progress Notes/Past History/Organ Systems/Hematology/Oncology (R)/Cancer/Cancer-Primary Site/stomach', 'notes/Progress Notes/Past History/Organ Systems/Hematology/Oncology (R)/Cancer/Cancer-Primary Site/bile duct', 'notes/Progress Notes/Past History/Organ Systems/Hematology/Oncology (R)/Cancer/Cancer-Primary Site/kidney', 'notes/Progress Notes/Past History/Organ Systems/Hematology/Oncology (R)/Cancer/Cancer-Primary Site/unknown', 'notes/Progress Notes/Past History/Organ Systems/Hematology/Oncology (R)/Cancer Therapy/Radiation Therapy within past 6 months/primary site', 'notes/Progress Notes/Past History/Organ Systems/Hematology/Oncology (R)/Cancer/Cancer-Primary Site/breast', 'notes/Progress Notes/Past History/Organ Systems/Hematology/Oncology (R)/Cancer/Cancer-Primary Site/uterus', 'notes/Progress Notes/Past History/Organ Systems/Hematology/Oncology (R)/Cancer Therapy/Radiation Therapy within past 6 months/bone', 'notes/Progress Notes/Past History/Organ Systems/Hematology/Oncology (R)/Cancer/Cancer-Primary Site/prostate', 'notes/Progress Notes/Past History/Organ Systems/Hematology/Oncology (R)/Cancer/Cancer-Primary Site/liver', 'notes/Progress Notes/Past History/Organ Systems/Hematology/Oncology (R)/Cancer/Cancer-Primary Site/pancreas - adenocarcinoma', 'notes/Progress Notes/Past History/Organ Systems/Hematology/Oncology (R)/Cancer/Cancer-Primary Site/ovary', 'notes/Progress Notes/Past History/Organ Systems/Hematology/Oncology (R)/Cancer Therapy/Radiation Therapy within past 6 months/other', 'notes/Progress Notes/Past History/Organ Systems/Hematology/Oncology (R)/Cancer/Cancer-Primary Site/sarcoma', 'notes/Progress Notes/Past History/Organ Systems/Hematology/Oncology (R)/Cancer Therapy/Chemotherapy/chemotherapy within past mo.', 'notes/Progress Notes/Past History/Organ Systems/Hematology/Oncology (R)/Cancer/Cancer-Primary Site/other', 'notes/Progress Notes/Past History/Organ Systems/Hematology/Oncology (R)/Cancer Therapy/Chemotherapy/Alkylating agents (bleomycin, cytoxan, cyclophos.)', 'notes/Progress Notes/Past History/Organ Systems/Hematology/Oncology (R)/Cancer/Cancer-Primary Site/testes', 'notes/Progress Notes/Past History/Organ Systems/Hematology/Oncology (R)/Cancer/Cancer-Primary Site/lung', 'notes/Progress Notes/Past History/Organ Systems/Hematology/Oncology (R)/Cancer/Cancer-Primary Site/melanoma', 'notes/Progress Notes/Past History/Organ Systems/Hematology/Oncology (R)/Cancer Therapy/Radiation Therapy within past 6 months/nodes', 'notes/Progress Notes/Past History/Organ Systems/Hematology/Oncology (R)/Cancer Therapy/Chemotherapy/BMT within past 12 mos.', 'notes/Progress Notes/Past History/Organ Systems/Hematology/Oncology (R)/Cancer Therapy/Chemotherapy/Cis-platinum', 'notes/Progress Notes/Past History/Organ Systems/Hematology/Oncology (R)/Cancer Therapy/Radiation Therapy within past 6 months/liver', 'notes/Progress Notes/Past History/Organ Systems/Hematology/Oncology (R)/Cancer/Cancer-Primary Site/head and neck', 'notes/Progress Notes/Past History/Organ Systems/Hematology/Oncology (R)/Cancer/Cancer-Primary Site/esophagus', 'notes/Progress Notes/Past History/Organ Systems/Hematology/Oncology (R)/Cancer/Cancer-Primary Site/bladder', 'notes/Progress Notes/Past History/Organ Systems/Hematology/Oncology (R)/Cancer Therapy/Chemotherapy/chemotherapy within past 6 mos.', 'notes/Progress Notes/Past History/Organ Systems/Hematology/Oncology (R)/Cancer Therapy/Radiation Therapy within past 6 months/lung', 'notes/Progress Notes/Past History/Organ Systems/Hematology/Oncology (R)/Cancer/Cancer-Primary Site/none', 'notes/Progress Notes/Past History/Organ Systems/Hematology/Oncology (R)/Cancer/Cancer-Primary Site/pancreas - islet cell', 'notes/Progress Notes/Past History/Organ Systems/Hematology/Oncology (R)/Cancer/Cancer-Primary Site/colon', 'notes/Progress Notes/Past History/Organ Systems/Hematology/Oncology (R)/Cancer Therapy/Radiation Therapy within past 6 months/brain', 'notes/Progress Notes/Past History/Organ Systems/Hematology/Oncology (R)/Cancer Therapy/Chemotherapy/Vincristine', 'notes/Progress Notes/Past History/Organ Systems/Hematology/Oncology (R)/Cancer/Cancer-Primary Site/brain') THEN 2
      ELSE
      0
    END
      ) AS cancer2,
    MAX (CASE
        WHEN ph.pasthistorypath IN ( 'notes/Progress Notes/Past History/Organ Systems/Hematology/Oncology (R)/Cancer/Hematologic Malignancy/AML', 'notes/Progress Notes/Past History/Organ Systems/Hematology/Oncology (R)/Cancer/Hematologic Malignancy/ALL', 'notes/Progress Notes/Past History/Organ Systems/Hematology/Oncology (R)/Cancer/Hematologic Malignancy/CLL', 'notes/Progress Notes/Past History/Organ Systems/Hematology/Oncology (R)/Cancer/Hematologic Malignancy/CML', 'notes/Progress Notes/Past History/Organ Systems/Hematology/Oncology (R)/Cancer/Hematologic Malignancy/leukemia - other') THEN 2
      ELSE
      0
    END
      ) AS leukemia2,
    MAX (CASE
        WHEN ph.pasthistorypath IN ( 'notes/Progress Notes/Past History/Organ Systems/Hematology/Oncology (R)/Cancer/Hematologic Malignancy/non-Hodgkins lymphoma', 'notes/Progress Notes/Past History/Organ Systems/Hematology/Oncology (R)/Cancer/Hematologic Malignancy/Hodgkins disease') THEN 2
      ELSE
      0
    END
      ) AS lymphoma2,
    MAX (CASE
        WHEN ph.pasthistorypath IN( 'notes/Progress Notes/Past History/Organ Systems/Cardiovascular (R)/Myocardial Infarction/MI - within 5 years', 'notes/Progress Notes/Past History/Organ Systems/Cardiovascular (R)/Myocardial Infarction/MI - remote', 'notes/Progress Notes/Past History/Organ Systems/Cardiovascular (R)/Myocardial Infarction/MI - within 6 months', 'notes/Progress Notes/Past History/Organ Systems/Cardiovascular (R)/Myocardial Infarction/MI - date unknown', 'notes/Progress Notes/Past History/Organ Systems/Cardiovascular (R)/Myocardial Infarction/MI - within 2 years', 'notes/Progress Notes/Past History/Organ Systems/Cardiovascular (R)/Myocardial Infarction/multiple/multiple') THEN 1
      ELSE
      0
    END
      ) AS mi1,
    MAX (CASE
        WHEN ph.pasthistorypath IN ( 'notes/Progress Notes/Past History/Organ Systems/Cardiovascular (R)/Congestive Heart Failure/CHF - class I', 'notes/Progress Notes/Past History/Organ Systems/Cardiovascular (R)/Congestive Heart Failure/CHF - class II', 'notes/Progress Notes/Past History/Organ Systems/Cardiovascular (R)/Congestive Heart Failure/CHF - severity unknown', 'notes/Progress Notes/Past History/Organ Systems/Cardiovascular (R)/Congestive Heart Failure/CHF - class III', 'notes/Progress Notes/Past History/Organ Systems/Cardiovascular (R)/Congestive Heart Failure/CHF', 'notes/Progress Notes/Past History/Organ Systems/Cardiovascular (R)/Congestive Heart Failure/CHF - class IV') THEN 1
      ELSE
      0
    END
      ) AS chf1,
    MAX (CASE
        WHEN ph.pasthistorypath = 'notes/Progress Notes/Past History/Organ Systems/Cardiovascular (R)/Peripheral Vascular Disease/peripheral vascular disease' THEN 1
      ELSE
      0
    END
      ) AS pvd1,
    MAX (CASE
        WHEN ph.pasthistorypath IN ( 'notes/Progress Notes/Past History/Organ Systems/Neurologic/TIA(s)/TIA(s) - within 6 months', 'notes/Progress Notes/Past History/Organ Systems/Neurologic/TIA(s)/TIA(s) - within 2 years', 'notes/Progress Notes/Past History/Organ Systems/Neurologic/TIA(s)/TIA(s) - remote', 'notes/Progress Notes/Past History/Organ Systems/Neurologic/TIA(s)/TIA(s) - within 5 years', 'notes/Progress Notes/Past History/Organ Systems/Neurologic/TIA(s)/multiple/multiple', 'notes/Progress Notes/Past History/Organ Systems/Neurologic/TIA(s)/TIA(s) - date unknown') THEN 1
      ELSE
      0
    END
      ) AS tia1,
    MAX (CASE
        WHEN ph.pasthistorypath = 'notes/Progress Notes/Past History/Organ Systems/Neurologic/Dementia/dementia' THEN 1
      ELSE
      0
    END
      ) AS dementia1,
    MAX (CASE
        WHEN ph.pasthistorypath IN ( 'notes/Progress Notes/Past History/Organ Systems/Pulmonary/COPD/COPD  - no limitations', 'notes/Progress Notes/Past History/Organ Systems/Pulmonary/COPD/COPD  - moderate', 'notes/Progress Notes/Past History/Organ Systems/Pulmonary/COPD/COPD  - severe') THEN 1
      ELSE
      0
    END
      ) AS copd1,
    MAX (CASE
        WHEN ph.pasthistorypath IN ( 'notes/Progress Notes/Past History/Organ Systems/Rheumatic/SLE/SLE', 'notes/Progress Notes/Past History/Organ Systems/Rheumatic/Rheumatoid Arthritis/rheumatoid arthritis', 'notes/Progress Notes/Past History/Organ Systems/Rheumatic/Scleroderma/scleroderma', 'notes/Progress Notes/Past History/Organ Systems/Rheumatic/Vasculitis/vasculitis', 'notes/Progress Notes/Past History/Organ Systems/Rheumatic/Dermato/Polymyositis/dermatomyositis') THEN 1
      ELSE
      0
    END
      ) AS ctd1,
    MAX (CASE
        WHEN ph.pasthistorypath IN ( 'notes/Progress Notes/Past History/Organ Systems/Gastrointestinal (R)/Peptic Ulcer Disease/peptic ulcer disease', 'notes/Progress Notes/Past History/Organ Systems/Gastrointestinal (R)/Peptic Ulcer Disease/peptic ulcer disease with h/o GI bleeding', 'notes/Progress Notes/Past History/Organ Systems/Gastrointestinal (R)/Peptic Ulcer Disease/hx GI bleeding/no') THEN 1
      ELSE
      0
    END
      ) AS pud1,
    MAX (CASE
        WHEN ph.pasthistorypath IN ( 'notes/Progress Notes/Past History/Organ Systems/Gastrointestinal (R)/Cirrhosis/clinical diagnosis', 'notes/Progress Notes/Past History/Organ Systems/Gastrointestinal (R)/Cirrhosis/biopsy proven') THEN 1
      ELSE
      0
    END
      ) AS liver1,
    CASE
      WHEN s.age LIKE '>%89' THEN 5
      WHEN s.age LIKE '' THEN 0
      WHEN CAST(s.age AS numeric) BETWEEN 80 AND 89 THEN 4
      WHEN CAST(s.age AS numeric) BETWEEN 70
    AND 79 THEN 3
      WHEN CAST(s.age AS numeric) BETWEEN 60 AND 69 THEN 2
      WHEN CAST(s.age AS numeric) BETWEEN 50
    AND 59 THEN 1
    ELSE
    0
  END
    AS age_score_charlson
  FROM
    `physionet-data.eicu_crd.patient` s
  LEFT JOIN
    `physionet-data.eicu_crd.pasthistory` ph
  ON
    s.patientunitstayid = ph.patientunitstayid
  GROUP BY
    s.patientunitstayid,
    s.age
  ORDER BY
    s.patientunitstayid )
SELECT
  t1.*,
  (t1.mets6+t1.aids6+t1.liver3+t1.stroke2+t1.renal2+t1.dm+t1.cancer2+t1.leukemia2+t1.lymphoma2+t1.mi1+ t1.chf1+t1.pvd1+t1.tia1+t1.dementia1+t1.copd1+t1.ctd1+t1.pud1+t1.liver1 + t1.age_score_charlson)AS final_charlson_score
FROM
  t1
ORDER BY
  t1.patientunitstayid

Query is running:   0%|          |

In [5]:
vent_df.to_csv('ventilation.csv')
cci.to_csv('charlson.csv')